In [1]:
import os
# Changing the working directory
os.chdir('../..')
os.getcwd()

'/home/javiermunoz/TFM_DSBD'

In [22]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("figure", figsize=(15, 5))

import os 

In [23]:
gpu_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from src.config import DATA_PATH, SPLITS
import pathlib

In [5]:
subset_size = 10
subset = f"subset_{subset_size}_lsa_64"

In [6]:
subset_paths = {}
for sp in SPLITS:
    subset_paths[sp] = pathlib.Path(os.path.join(DATA_PATH, subset, sp))

## 2. Loading Video Dataset

https://www.tensorflow.org/tutorials/load_data/video

In [7]:
from load_dataset_aux import frames_from_video_file, to_gif
from frame_generator import FrameGenerator
import tensorflow as tf

In [8]:
fg = FrameGenerator(subset_paths['train'], 10, training=True)

frames, label = next(fg())
print(f"Shape: {frames.shape}")
print(f"Label: {label}")

Shape: (10, 224, 224, 3)
Label: 1


In [15]:
n_frames = 10
batch_size = 6

# We specify the shape of the output, such as, the Generator will produce a tuple of both
# videos, class, where the videos will have 3 channels, and the rest of dimensions will remain the same.
#   VideoShape -> (F, H, W, C)
output_signature = (tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.int16))

train_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['train'], n_frames, training=True),
                                          output_signature=output_signature)
# Also, we batchify the data, so the training process is not as memory consuming as if the whole dataset was 
# loaded into memory.
# VideoShape -> (B, F, H, W, C)
# train_ds = train_ds.batch(batch_size)
train_ds = train_ds.batch(batch_size)

# We reproduce this process for the validation and test splits too.
val_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['val'], n_frames),
                                        output_signature=output_signature)

val_ds = val_ds.batch(batch_size)
test_ds = tf.data.Dataset.from_generator(FrameGenerator(subset_paths['test'], n_frames),
                                         output_signature=output_signature)

test_ds = test_ds.batch(batch_size)

In [16]:
CACHE_FLAG = True
if CACHE_FLAG:    
    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
    val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [17]:
# Print the shapes of the data
train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

Shape of training set of frames: (6, 10, 224, 224, 3)
Shape of training labels: (6,)
Shape of validation set of frames: (6, 10, 224, 224, 3)
Shape of validation labels: (6,)


In [18]:
(None, *train_frames.shape[2:])

(None, 224, 224, 3)

# 3. Model definition and Training

In [19]:
model = tf.keras.Sequential([
    # Construct the input layer with no definite frame size.
    tf.keras.layers.InputLayer(input_shape=(None, *train_frames.shape[2:])),
    # We will construct 3 `ConvLSTM2D` layers with batch normalization,
    tf.keras.layers.ConvLSTM2D(
        filters=8,
        kernel_size=(3, 3),
        strides=(1,1),
        stateful=False,
        return_sequences=True,
        activation="relu",
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(
        filters=16,
        kernel_size=(3, 3),
        strides=(2,2),
        stateful=False,
        return_sequences=True,
        activation="relu",
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(
        filters=32,
        kernel_size=(3, 3),
        strides=(1,1),
        stateful=False,
        return_sequences=True,
        activation="relu",
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ConvLSTM2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(2,2),
        stateful=False,
        return_sequences=False,
        activation="relu",
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(3,3),                       
        strides=(2,2),
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Conv2D(
        filters=16,
        kernel_size=(3,3),                       
        strides=(2,2),
    ),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=10, activation='sigmoid')
])
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy, 
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['acc']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_4 (ConvLSTM2D)  (None, None, 222, 222, 8  3200      
                             )                                   
                                                                 
 batch_normalization_6 (Batc  (None, None, 222, 222, 8  32       
 hNormalization)             )                                   
                                                                 
 conv_lstm2d_5 (ConvLSTM2D)  (None, None, 110, 110, 1  13888     
                             6)                                  
                                                                 
 batch_normalization_7 (Batc  (None, None, 110, 110, 1  64       
 hNormalization)             6)                                  
                                                                 
 conv_lstm2d_6 (ConvLSTM2D)  (None, None, 108, 108, 3 

In [20]:
# Define some callbacks to improve training.
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 20

# Fit the model to the training data using a generator.
model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=[early_stopping, reduce_lr],
)

Epoch 1/20


InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_27' defined at (most recent call last):
    File "/home/javiermunoz/miniconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/javiermunoz/miniconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/javiermunoz/miniconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/javiermunoz/miniconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/javiermunoz/miniconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/javiermunoz/miniconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/javiermunoz